In [1]:
import sys
import pandas as pd
import json
import os
import pickle
import re
import difflib
from difflib import SequenceMatcher
import jellyfish
pd.set_option('display.max_columns', 100)

In [2]:
import spotipy
import spotipy.util as util

In [4]:
user_name = "Pillsbury Darboy"
client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
client_secret = "53634fc86e8a4f71b7459197084e3cdf"
redirect_uri = "http://localhost/"
scope = 'user-library-read'
from spotipy.util import prompt_for_user_token
token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
from spotipy import Spotify
spotify = Spotify(auth=token)

In [5]:
class SetlistGenerator_Spotify(object):

    def __init__(self, seed_tracks='', seed_genres='', seed_artists=''):
        self.seed_genres = seed_genres
        self.seed_tracks = seed_tracks
        self.seed_artists = seed_artists
        
        self.AuthorizeSpotify()
        
        #Setlist starts with seed_track
        self.set_list = self.get_track_metadata_single()
        
        self.genres = ['chicago-house', 'club', 'dancehall', 'deep-house', 'detroit-techno', 'dub', 'edm',
         'electro', 'electronic', 'groove', 'house', 'idm', 'industrial', 'minimal-techno', 'progressive-house', 
          'soul', 'techno', 'trance']
        self.moods = ['happy', 'sad']
        
        self.track_pool = self.getRecommendations()
        
    def getRecommendations(self):
        # Query spotify for recommendations based on a track
        recommendations = self.spotify.recommendations(seed_artists=self.seed_artists, 
                                                       seed_genres=self.seed_genres, 
                                                       seed_tracks=self.seed_tracks, 
                                                       limit = 50)
        # Convert spotify data into readable format
        metadata = self.get_track_metadata(recommendations)
        # Retrieve audio features for spotify data
        audio_features = pd.DataFrame(self.spotify.audio_features(self.get_track_ids(recommendations)))
        df = metadata.merge(audio_features)
        
        #Correct key + mode data 
        df = self.key_mode_convert(df)
        #del df['type']
        #del df['analysis_url']
        #del df['disc_number']
        #del df['track_number']
        return(df)

    def getNextTrack(self, method = 0, min_dance=0.6):
        candidates = self.track_pool
        current_song_key = int(self.set_list[-1:]['camelot'])
        if method == 0:
            candidates = self.filter_by_camelot(df = candidates, 
                                                song_key_camelot = current_song_key)
            candidates = self.filter_by_popularity(candidates)
            candidates = self.filter_by_danceability(candidates, min_dance=min_dance)
        else:
            pass
        self.set_list = self.set_list.append(candidates.sample(1))
        self.set_list = self.set_list.reset_index(drop=True)
        
        return(self.set_list)
    
    ############################################
    ######      SEED     UTILITIES        ######
    ############################################
    
    def changeSeedTracks(self, use_last = 2):
        self.seed_tracks = list(self.set_list['id'][-use_last:])
        
    def changeSeedGenres(self, new_genres):
        self.seed_genres = new_genres
        
    def changeSeedArtists(self, new_artists):
        self.seed_genres = artists
        
    ############################################
    ######    NARRATIVE     UTILITIES     ######
    ############################################
    
    def CreateTurnarounds(self, **kwargs):
        for key, value in kwargs.items():
            print(key, value)
        self.narrative_valence  = np.random()
        self.narrative_energy  = np.random()
        self.narrative_mode  = np.random()
        self.narrative_tempo  = np.random()
        self.narrative_popularity  = np.random()
        self.narrative_danceability  = np.random()
        self.narrative_genre = np.random()
        
    def CreateNarratives(self, **kwargs):
        for key, value in kwargs.items():
            print(key, value)
        self.narrative_valence  = np.random()
        self.narrative_energy  = np.random()
        self.narrative_mode  = np.random()
        self.narrative_tempo  = np.random()
        self.narrative_popularity  = np.random()
        self.narrative_danceability  = np.random()
        self.narrative_genre = np.random()
        
    def NarrativeUtilDanceability(self):
        pass
    
    def NarrativeUtilMode(self):
        pass
    
    def NarrativeUtilPopularity(self):
        pass
    
    def NarrativeUtilEnergy(self):
        pass
    
    def NarrativeUtilValence(self):
        pass
    
    ############################################
    ######     TRACK   POOL   UTILITIES   ######
    ############################################
        
    def getTrackPool(self):
        return self.track_pool
    
    def expandTrackPool(self):
        self.track_pool = self.track_pool.append(self.getRecommendations())
        self.cleanTrackPool()
        
        print('Expanded track pool')
    
    def cleanTrackPool(self):
        self.track_pool = self.track_pool[-self.track_pool['id'].isin(list(self.set_list['id']))]
        self.track_pool = self.track_pool.drop_duplicates()
        self.track_pool = self.track_pool.reset_index(drop=True)

        
    ############################################
    ######    TRACK SELECTION UTILITIES   ######
    ############################################
        
    def closest_keys(self, camelot):
        closest = []
        closest.append(camelot)
        closest.append((camelot + 1)%12)
        closest.append((camelot - 1)%12)
        return(closest)
    
    def filter_by_camelot(self, df, song_key_camelot):
        filtered = []
        for i in self.closest_keys(song_key_camelot):
            filtered.append(pd.DataFrame(df[df['camelot'] == i]))
        return(pd.concat(filtered))
        
    def filter_by_popularity(self, df, min_pop=0, max_pop=100):
        df = df[df['popularity'] >= min_pop]
        df = df[df['popularity'] <= max_pop]
        return(df)
    
    def filter_by_tempo(self, df, min_tempo = 90, max_tempo = 145):
        df = df[df['tempo'] >= min_tempo]
        df = df[df['tempo'] <= max_tempo]
        return(df)
    
    def filter_by_valence(self, df=None, min_valence = 0, max_valence = 1):
        df = self.track_pool
        df = df[df['valence'] >= min_valence]
        df = df[df['valence'] <= max_valence]
        return(df)
    
    def filter_by_danceability(self, df=None, min_dance = 0, max_dance = 1):
        df = self.track_pool
        df = df[df['danceability'] >= min_dance]
        df = df[df['danceability'] <= max_dance]
        return(df)
    
    def filter_by_energy(self, df=None, min_energy = 0, max_energy = 1):
        df = self.track_pool
        df = df[df['energy'] >= min_energy]
        df = df[df['energy'] <= max_energy]
        return(df)
    
    def filter_by_liveness(self, df=None, min_liveness = 0, max_liveness = 1):
        df = self.track_pool
        df = df[df['liveness'] >= min_liveness]
        df = df[df['liveness'] <= max_liveness]
        return(df)
    
    def filter_by_acousticness(self, df=None, min_acoustic = 0, max_acoustic = 1):
        df = self.track_pool
        df = df[df['acousticness'] >= min_acoustic]
        df = df[df['acousticness'] <= max_acoustic]
        return(df)
    
    def filter_by_instrumentalness(self, df=None,
                                   min_instrumentalness = 0, 
                                   max_instrumentalness = 1):
        df = self.track_pool
        df = df[df['instrumentalness'] >= min_instrumentalness]
        df = df[df['instrumentalness'] <= max_instrumentalness]
        return(df)



    ############################################
    ######        METADATA UTILITIES      ######
    ############################################
    
    def get_track_metadata(self, recommendations):
        tracklist_frames = []
        for track in recommendations['tracks']:
            tracklist_frames.append({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']})
        return(pd.DataFrame(tracklist_frames))

    def get_track_metadata_single(self):
        track_id = self.seed_tracks[0]
        track = self.spotify.track(track_id)
        metadata = pd.DataFrame({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']}, index =[0])
        audio_features = pd.DataFrame(self.spotify.audio_features(track_id))
        df = metadata.merge(audio_features)
        df = self.key_mode_convert(df)
        return(df)
    
    def get_track_ids(self, recommendations):
        id_list = []
        for spotify_id in recommendations['tracks']:
            id_list.append(spotify_id['id'])
        return(id_list)
    
    def key_mode_convert(self, recommendations):
        df = recommendations
        camelot_list = []
        for i in range(0, len(df)):
            if df['mode'][i] == 1:
                if (df['key'][i] == 0):
                    camelot_list.append(7)
                if (df['key'][i] == 1):
                    camelot_list.append(2)
                if (df['key'][i] == 2):
                    camelot_list.append(9)
                if (df['key'][i] == 3):
                    camelot_list.append(4)
                if (df['key'][i] == 4):
                    camelot_list.append(11)                
                if (df['key'][i] == 5):
                    camelot_list.append(6)                
                if (df['key'][i] == 6):
                    camelot_list.append(1)                
                if (df['key'][i] == 7):
                    camelot_list.append(8)                
                if (df['key'][i] == 8):
                    camelot_list.append(3)                
                if (df['key'][i] == 9):
                    camelot_list.append(10)                
                if (df['key'][i] == 10):
                    camelot_list.append(5)
                if (df['key'][i] == 11):
                    camelot_list.append(0)
            else:
                if (df['key'][i] == 0):
                    camelot_list.append(4)
                if (df['key'][i] == 1):
                    camelot_list.append(11)
                if (df['key'][i] == 2):
                    camelot_list.append(6)
                if (df['key'][i] == 3):
                    camelot_list.append(1)
                if (df['key'][i] == 4):
                    camelot_list.append(8)                
                if (df['key'][i] == 5):
                    camelot_list.append(3)                
                if (df['key'][i] == 6):
                    camelot_list.append(10)                
                if (df['key'][i] == 7):
                    camelot_list.append(5)                
                if (df['key'][i] == 8):
                    camelot_list.append(0)                
                if (df['key'][i] == 9):
                    camelot_list.append(7)                
                if (df['key'][i] == 10):
                    camelot_list.append(2)
                if (df['key'][i] == 11):
                    camelot_list.append(9)
        df['camelot'] = camelot_list
        return(df)

    ############################################
    ######        SPOTIFY  UTILITIES      ######
    ############################################
    
    def SearchSpotify(self, search_string):
        return self.spotify.search(search_string)
    

    
    def AuthorizeSpotify(self):
        print('Authorizing Spotify')
        self.user_name = "Ronald Marp"
        self.client_id = '1e05e910ef664f27949005267fdd0936'
        self.client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
        self.redirect_uri = "http://localhost/"
        self.scope = 'user-library-read'
        self.token = prompt_for_user_token(username=self.user_name,scope=self.scope,
                                           client_id=self.client_id,
                                           client_secret=self.client_secret,
                                           redirect_uri=self.redirect_uri)
        self.spotify = Spotify(auth=self.token)

In [6]:
## Percolator Meme - Teeth ['2QT5afoBT6nQabNuvcxXUa']

In [7]:
def search_for_artistID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]['artists'][0]
    print('Found artist %s' %result['name'])
    return(result['id'])

In [8]:
def search_for_trackID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]
    print('Found track %s by %s' %(result['name'], result['artists'][0]['name']))
    return(result['id'])

In [9]:
seed_artist_id = search_for_artistID('Aleksi Perala')

Found artist Aleksi Perälä


In [10]:
seed_track_id = search_for_trackID('Eckograph Hubot')

Found track Eckograph - Original Mix by Hubot


In [13]:
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id])
                             #seed_artists=[seed_artist_id], 
                             #seed_genres=['minimal-techno', 'techno']) #seed_genres=['minimal-techno', 'techno', 'detroit-techno'])

Authorizing Spotify


In [28]:
def GenerateSetList(setlist_generator, setlength = 10, methods = 1):
    d = setlist_generator
    while len(d.set_list) <= setlength:
        d.getNextTrack()
        d.changeSeedTracks()
        d.expandTrackPool()
    return(d)
    

In [14]:
while len(d.set_list) < 10:
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool


,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
0,Triadz,Hubot,1,415633,35ug0mCea5X0LX7hxN0xQ2,0,Eckograph - Original Mix,3,0.019800,https://api.spotify.com/v1/audio-analysis/35ug...,0.718,415634,0.955,0.908,4,0.1090,-9.657,0,0.0541,125.030,4,https://api.spotify.com/v1/tracks/35ug0mCea5X0...,audio_features,spotify:track:35ug0mCea5X0LX7hxN0xQ2,0.0398,8
1,Silky Squeak,Nuria Ghia,1,437816,23B37qp4iUYGomS6a3NxYt,0,Silky Squeak - Original Mix,1,0.015600,https://api.spotify.com/v1/audio-analysis/23B3...,0.750,437816,0.801,0.430,7,0.1180,-8.264,1,0.0831,125.002,4,https://api.spotify.com/v1/tracks/23B37qp4iUYG...,audio_features,spotify:track:23B37qp4iUYGomS6a3NxYt,0.2160,8
2,Ambiguous Relation EP,Héctor Oaks,1,364038,3BCXVpSFipkqheGilNG4dB,1,Revolt - I/Y Remix,5,0.065100,https://api.spotify.com/v1/audio-analysis/3BCX...,0.787,364039,0.878,0.916,2,0.1210,-11.709,1,0.0672,127.998,4,https://api.spotify.com/v1/tracks/3BCXVpSFipkq...,audio_features,spotify:track:3BCXVpSFipkqheGilNG4dB,0.0667,9
3,Silky Squeak,Nuria Ghia,1,434021,1I4k2KXzpC8NmdJ9fq8EMP,0,Jack Soul - Original Mix,3,0.112000,https://api.spotify.com/v1/audio-analysis/1I4k...,0.842,434021,0.950,0.826,1,0.1040,-7.719,1,0.0617,125.021,4,https://api.spotify.com/v1/tracks/1I4k2KXzpC8N...,audio_features,spotify:track:1I4k2KXzpC8NmdJ9fq8EMP,0.6660,2
4,Different Samba,Quenum,1,410021,2P1CPmgIHskPQZrWGSa8Ng,0,Fatty Boy,3,0.000088,https://api.spotify.com/v1/audio-analysis/2P1C...,0.759,410022,0.857,0.845,7,0.1100,-10.396,1,0.0357,122.014,4,https://api.spotify.com/v1/tracks/2P1CPmgIHskP...,audio_features,spotify:track:2P1CPmgIHskPQZrWGSa8Ng,0.3350,8
5,Martyr,Jamie React,1,539289,2R56xjy815FVM4RjJJMEHw,0,Martyr - Jnr Remix,4,0.000087,https://api.spotify.com/v1/audio-analysis/2R56...,0.805,539289,0.703,0.872,2,0.1080,-10.273,1,0.0916,124.980,4,https://api.spotify.com/v1/tracks/2R56xjy815FV...,audio_features,spotify:track:2R56xjy815FVM4RjJJMEHw,0.0938,9
6,Always a Mystery,Alex Grekov,1,449989,7bGKrJmCDZaiI4vCZxzrEv,0,Always a Mystery - Alex Grekov Remix,3,0.002750,https://api.spotify.com/v1/audio-analysis/7bGK...,0.759,449990,0.907,0.720,11,0.0828,-9.077,0,0.0717,122.008,4,https://api.spotify.com/v1/tracks/7bGKrJmCDZai...,audio_features,spotify:track:7bGKrJmCDZaiI4vCZxzrEv,0.6280,9
7,Ominous,Protocluster,1,420983,2FliYwUbWbpta6QdmccuE0,0,Silent Nights - Original Mix,4,0.000681,https://api.spotify.com/v1/audio-analysis/2Fli...,0.641,420984,0.669,0.141,4,0.1020,-8.029,1,0.1370,122.019,3,https://api.spotify.com/v1/tracks/2FliYwUbWbpt...,audio_features,spotify:track:2FliYwUbWbpta6QdmccuE0,0.4380,11
8,002,Talismann,1,454850,13mVUFcuUt7cwMFnqgAOvV,9,Zula,2,0.092700,https://api.spotify.com/v1/audio-analysis/13mV...,0.827,454850,0.743,0.945,6,0.1080,-12.979,0,0.2350,129.008,4,https://api.spotify.com/v1/tracks/13mVUFcuUt7c...,audio_features,spotify:track:13mVUFcuUt7cwMFnqgAOvV,0.1410,10
9,Crossed Paths,Shifted,1,262506,08ThqBtTZ24pUhyVGwP3E7,7,Colour Of The Fall,6,0.191000,https://api.spotify.com/v1/audio-analysis/08Th...,0.761,262507,0.924,0.910,10,0.1030,-11.727,0,0.1310,126.036,4,https://api.spotify.com/v1/tracks/08ThqBtTZ24p...,audio_features,spotify:track:08ThqBtTZ24pUhyVGwP3E7,0.4060,2


In [39]:
d.set_list['uri'].to_csv('../../../../spotify-downloader/output.txt', index=False)

In [26]:
d.set_list.duplicated(subset=['track', 'artist'])

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [ ]:
len(mixesdb)

In [ ]:
# Load and Process Datasets for big-data generation

spotify = pd.read_csv('data/spotify/spotify_database_mergedV2.csv')
del spotify['Unnamed: 0']
del spotify['Unnamed: 0.1']
spotify['artist_and_track'] = spotify['artist'] + ' ' + spotify['track']
print("Read and processed Spotify data")

discogs_m = pd.read_csv('data/discogs/discogs_mixed.csv')
discogs_r = pd.read_csv('data/discogs/discogs_releases.csv')
to_concat = [discogs_m, discogs_r]
discogs = pd.concat(to_concat)
del discogs_m
del discogs_r
del discogs['Unnamed: 0']
del discogs['genre']
del discogs['labels']
del discogs['length']
discogs['artist'] = discogs['artist'].apply(lambda x: re.sub(pattern='\([0-9]*\)', string=x, repl = '').strip())
discogs['artist_and_track'] = discogs['artist'] + ' ' + discogs['track']
print("Read and processed Discogs data")

mixesdb = pd.read_csv('data/mixesdb/mixesdb_all_sets_featurized.csv')
del mixesdb['Unnamed: 0']
mixesdb['artist_and_track'] = mixesdb['artist'] + ' ' + mixesdb['track']
print("Read and processed MixesDB data")


In [ ]:
spotify['popularity'].max()